In [35]:
import pandas as pd
import glob
import subprocess
import os
import time
import psutil
import time
from typing import NamedTuple

## Notebook Utils

In [ ]:
class Results(NamedTuple):
    """Container for storing TVC or haplotype caller results."""
    sample_id: str
    runtime_minutes: float
    memory_mb: float
    cores_used: int
    vcf_path: str

In [38]:
class BenchmarkResult(NamedTuple):
    """Container for storing benchmarking results."""
    sample_id: str
    snp_precision: float
    snp_recall: float
    snp_f1: float
    indel_precision: float
    indel_recall: float
    indel_f1: float

In [40]:

# VCF and Benchmark regions from https://www.nist.gov/programs-projects/genome-bottle version v4.2.1
BENCHMARK_VCF = "HG001_GRCh38_1_22_v4.2.1_benchmark.vcf.gz"
BENCHMARK_REGIONS = "HG001_GRCh38_1_22_v4.2.1_benchmark.bed"
REFERENCE_GENOME = "hg38.fa"

In [ ]:
# Specify the number of cores, 90 is recommended. 
CORES = 90

In [ ]:
def run_TVC(input_bam, input_ref, cores):
    """
    Run TVC variant caller and monitor resource usage across all threads/processes.
    
    :param input_bam: Path to input BAM file.
    :param input_ref: Path to reference genome file.
    :param cores: Number of CPU cores to use.
    :return: TVCResult containing runtime and memory usage information.
    """
    output_vcf = input_bam.replace(".bam", ".tvc.vcf")
    cmd = f"target/debug/tvc -t {cores} {input_ref} {input_bam} {output_vcf}"

    start_time = time.time()
    proc = subprocess.Popen(cmd, shell=True)
    ps_proc = psutil.Process(proc.pid)

    peak_memory = 0
    while proc.poll() is None:
        try:
            # Include child processes’ memory
            mem_total = ps_proc.memory_info().rss
            for child in ps_proc.children(recursive=True):
                mem_total += child.memory_info().rss
            peak_memory = max(peak_memory, mem_total)
        except psutil.NoSuchProcess:
            break
        time.sleep(0.05)

    end_time = time.time()
    elapsed_minutes = (end_time - start_time) / 60

    result = Results(
        sample_id=input_bam.split(".")[0],
        runtime_minutes=elapsed_minutes,
        memory_mb=peak_memory / (1024 ** 2),
        vcf_path=output_vcf,
        cores_used=cores
    )
    return result


In [ ]:
def run_hap_caller(input_bam, input_ref, cores):
    """
    Run haplotype caller variant caller and monitor resource usage across all threads/processes.
    
    :param input_bam: Path to input BAM file.
    :param input_ref: Path to reference genome file.
    :param cores: Number of CPU cores to use.
    :return: Results containing runtime and memory usage information.
    """
    output_vcf = input_bam.replace(".bam", ".hap.vcf")
    cmd = f"gatk HaplotypeCaller -t {cores} -R {input_ref} -I {input_bam} -O {output_vcf}"

    start_time = time.time()
    proc = subprocess.Popen(cmd, shell=True)
    ps_proc = psutil.Process(proc.pid)

    peak_memory = 0
    while proc.poll() is None:
        try:
            # Include child processes’ memory
            mem_total = ps_proc.memory_info().rss
            for child in ps_proc.children(recursive=True):
                mem_total += child.memory_info().rss
            peak_memory = max(peak_memory, mem_total)
        except psutil.NoSuchProcess:
            break
        time.sleep(0.05)

    end_time = time.time()
    elapsed_minutes = (end_time - start_time) / 60

    result = Results(
        sample_id=input_bam.split(".")[0],
        runtime_minutes=elapsed_minutes,
        memory_mb=peak_memory / (1024 ** 2),
        vcf_path=output_vcf,
        cores_used=cores
    )
    return result

In [42]:
def extract_accuracy_metrics(benchmark_vcf, test_vcf, regions_bed, reference_genome):
    """
    Extract accuracy metrics using hap.py Docker container.
    :param benchmark_vcf: Path to benchmark VCF file.
    :param test_vcf: Path to test VCF file.
    :param regions_bed: Path to BED file defining regions of interest.
    :param reference_genome: Path to reference genome file.
    :return: BenchmarkResult containing precision, recall, and F1 scores for SNPs and INDELs.
    """
    random_suffix = str(os.getpid())
    docker_image = "quay.io/biocontainers/hap.py:0.3.14--py27h5c5a3ab_0"
    cmd = f"docker run --rm -v {os.getcwd()}:/data {docker_image} hap.py /data/{benchmark_vcf} /data/{test_vcf} -r /data/{reference_genome} -f /data/{regions_bed} -o /data/temp_happy_output.{random_suffix} --threads 90"
    subprocess.run(cmd, shell=True, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
    metrics_file = f'temp_happy_output.{random_suffix}.summary.csv'

    df = pd.read_csv(metrics_file)
    for row in df.itertuples():
        if row.Filter != 'PASS':
            continue
        if row.Type == 'SNP':
            snp_precision = row._12
            snp_recall = row._11
            snp_f1 = row._14
        elif row.Type == 'INDEL':
            indel_precision = row._12
            indel_recall = row._11
            indel_f1 = row._14

    os.remove(metrics_file)

    
    result = BenchmarkResult(
        sample_id=test_vcf.split(".")[0],
        snp_precision=snp_precision,
        snp_recall=snp_recall,
        snp_f1=snp_f1,
        indel_precision=indel_precision,
        indel_recall=indel_recall,
        indel_f1=indel_f1
    )
    return result

## Analysis on Taps and Unconverted

In [ ]:
input_bams = glob.glob("*.deduped.bam")
tvc_results = []
accuracy_results = []
for bam in input_bams:
    tvc_result = run_TVC(bam, REFERENCE_GENOME, cores=90)
    accuracy_results.append(extract_accuracy_metrics(BENCHMARK_VCF, tvc_result.vcf_path, BENCHMARK_REGIONS, REFERENCE_GENOME))
    tvc_results.append(tvc_result)


In [ ]:
tvc_df = pd.DataFrame(tvc_results)
tvc_df = tvc_df.set_index('sample_id')
accuracy_df = pd.DataFrame(accuracy_results)
accuracy_df = accuracy_df.set_index('sample_id')

final_df = pd.concat([tvc_df, accuracy_df], axis=1)
final_df.to_csv("tvc_benchmarking_results.tsv", sep="\t")

## Results on demo data

In [ ]:
demo_tvc_results = run_TVC("999999.bam", REFERENCE_GENOME, cores=90)
demo_happy_results = extract_accuracy_metrics(BENCHMARK_VCF, demo_tvc_results.vcf_path, BENCHMARK_REGIONS, REFERENCE_GENOME)


/tmp/ipykernel_1467351/2730702165.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  depth_of_coverage = int(wgs_data_csv['MEDIAN_COVERAGE'])


In [ ]:
demo_tvc_df = pd.DataFrame([demo_tvc_results])
demo_happy_df = pd.DataFrame([demo_happy_results])

demo_final_df = pd.concat([demo_tvc_df, demo_happy_df], axis=1)
demo_final_df.to_csv("demo_tvc_benchmarking_results.tsv", sep="\t")

## Below are results on the demo set

In [ ]:
demo_final_df

## Run haplotype caller for comparison
### Make sure GATK is intalled first...
Intallation instructions can be found here: 

https://gatk.broadinstitute.org/hc/en-us/articles/360041320571--How-to-Install-all-software-packages-required-to-follow-the-GATK-Best-Practices

In [ ]:
hap_caller_results = []
hap_caller_accuracy_results = []
for bam in input_bams:
    hap_caller_result = run_hap_caller(bam, REFERENCE_GENOME, cores=90)
    hap_caller_accuracy_results.append(extract_accuracy_metrics(BENCHMARK_VCF, hap_caller_result.vcf_path, BENCHMARK_REGIONS, REFERENCE_GENOME))
    hap_caller_results.append(hap_caller_result)

In [ ]:
hap_caller_df = pd.DataFrame(hap_caller_results)
hap_caller_df = hap_caller_df.set_index('sample_id')
hap_caller_accuracy_df = pd.DataFrame(hap_caller_accuracy_results)
hap_caller_accuracy_df = hap_caller_accuracy_df.set_index('sample_id')

final_df = pd.concat([hap_caller_df, hap_caller_accuracy_df], axis=1)
final_df.to_csv("hap_caller_benchmarking_results.tsv", sep="\t")